In [ ]:
import pandas as pd
import plotly.express as px



df = pd.read_csv('/content/drive/MyDrive/Zepto/flipkart_com-ecommerce_sample.csv')
df = df.dropna(how='all')
brand_counts = df['brand'].value_counts()
fig = px.bar(x=brand_counts.index, y=brand_counts.values,
             labels={'x': 'Brand', 'y': 'Count'},
             title='Distribution of Brands',color_discrete_sequence=['#ff0000'])

fig.update_layout(xaxis_tickangle=45)
fig.write_html("/content/drive/MyDrive/Zepto/brand_distribution.html")
fig.show()



In [ ]:
import pandas as pd
from collections import Counter
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

all_descriptions = " ".join(df['description'].dropna().astype(str))

words = all_descriptions.lower().split()

stop_words = set(stopwords.words('english'))
stop_words.update(['rs.','-'])
words = [word for word in words if word not in stop_words]

word_counts = Counter(words)

top_words = word_counts.most_common(100)
print(len(top_words))
df_words = pd.DataFrame(top_words, columns=['Word', 'Count'])

fig = px.bar(df_words, x='Word', y='Count',
             labels={'x': 'Word', 'y': 'Frequency'},
             title='Top 100 Most Common Words in Descriptions')

fig.update_layout(xaxis_tickangle=45,
                  xaxis={'categoryorder':'total descending'},
                  xaxis_tickmode='linear',
                  xaxis_tickfont=dict(size=8))
fig.write_html("/content/drive/MyDrive/Zepto/common-words.html")
fig.show()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


100


In [29]:
# prompt: i need an average brand rating from all the products

import pandas as pd
import plotly.express as px
from collections import Counter
import nltk
from nltk.corpus import stopwords



df = pd.read_csv('/content/drive/MyDrive/Zepto/flipkart_com-ecommerce_sample.csv')

brand_counts = df['brand'].value_counts()



df_filtered = df[df['product_rating'] != 'No rating available']
df = df.dropna(how='all')
print(df["brand"].isna().sum())
print(df["description"].isna().sum())
df_filtered['product_rating'] = df_filtered['product_rating'].astype(float)
df[df['description'].isnull()]
average_ratings = df_filtered.groupby('brand')['product_rating'].mean()
average_ratings = average_ratings.sort_values(ascending=False)
print(average_ratings)

fig = px.bar(x=average_ratings.index, y=average_ratings.values,
             labels={'x': 'Brand', 'y': 'Average Rating'},
             title='Average Ratings by Brand')

fig.update_layout(xaxis_tickangle=45)
fig.write_html("/content/drive/MyDrive/Zepto/average_brand_ratings.html")
fig.show()

null_brands = df['brand'].isnull().sum()
non_null_brands = df['brand'].notnull().sum()

data = {'Brand Status': ['Branded', 'Unbranded'], 'Count': [null_brands, non_null_brands]}
df_pie = pd.DataFrame(data)

fig = px.pie(df_pie, values='Count', names='Brand Status',
             title='Branded vs. Unbranded Products')
fig.update_traces(textinfo='value+percent', textposition='inside')

fig.write_html("/content/drive/MyDrive/Zepto/brand_null.html")

fig.show()



5864
2
brand
Mark Home               5.0
Mandhania               5.0
Mast & Harbour          5.0
Max                     5.0
Metmo                   5.0
                       ... 
D-LINK                  1.0
Mz Nova                 1.0
Neutron                 1.0
WearYourShine by PCJ    1.0
PURPLE                  1.0
Name: product_rating, Length: 397, dtype: float64


In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [24]:
import pandas as pd
import re
from nltk import sent_tokenize
import numpy as np

def remove_marketing_phrases(text,remove_entire_sentence=False):
    marketing_phrases = [
        r'\bfree shipping\b',
        r'\blimited time offer\b',
        r'\bwhile supplies last\b',
        r'\bact now\b',
        r'\bfor a limited time\b',
        r'\bbest seller\b',
        r'\btop rated\b',
        r'\bgreat value\b',
        r'\bsatisfaction guaranteed\b',
        r'\bmoney-back guarantee\b',
        r'\bexclusive offer\b',
        r'\bnew arrival\b',
        r'\bspecial edition\b',
        r'Shop Online For Apparels. Huge Collection of Branded Clothes Only at Flipkart.com',
        r'(?i)for\s+rs\.?\s*\d+(?:\.\d+)?\s*only',  # Matches "for Rs.999 only", "for rs 999 only", etc.
        r'(?i)\bonly\s+for\s+rs\.?\s*\d+(?:\.\d+)?',  # Matches "only for Rs.999", "only for rs 999", etc.
        r'(?i)at\s+just\s+rs\.?\s*\d+(?:\.\d+)?',   # Matches "at just Rs.999", "at just rs 999", etc.
        r'(?i)price:\s*rs\.?\s*\d+(?:\.\d+)?',      # Matches "Price: Rs.999", "price: rs 999", etc.
        r'(?i)mrp:?\s*rs\.?\s*\d+(?:\.\d+)?',       # Matches "MRP: Rs.999", "mrp rs 999", etc.
        r'(?i)\brs\.?\s*\d+(?:\.\d+)?\s*only',      # Matches "Rs.999 only", "rs 999 only", etc.
        r'(?i)for\s+only\s+rs\.?\s*\d{1,3}(?:,\d{3})*(?:\.\d+)?(?:\s+online\s+in\s+india)?',
        r'\bcash on delivery\b',
        r'\bonly genuine products\b',
        r'\bonline in india\b',
    ]
    pattern = '|'.join(f'({phrase})' for phrase in marketing_phrases)

    if remove_entire_sentence:
        sentences = sent_tokenize(text)
        cleaned_sentences = [sent for sent in sentences if not re.search(pattern, sent, flags=re.IGNORECASE)]
        cleaned_text = ' '.join(cleaned_sentences)
    else:
        cleaned_text = re.sub(pattern, '', text, flags=re.IGNORECASE)

    cleaned_text = ' '.join(cleaned_text.split())


    return cleaned_text


def remove_duplicates(text):
    if pd.isna(text):
        return np.nan
    sentences = sent_tokenize(text)
    unique_sentences = []
    seen_sentences = set()
    for sentence in sentences:
        if sentence not in seen_sentences:
            unique_sentences.append(sentence)
            seen_sentences.add(sentence)
    cleaned_text = ' '.join(unique_sentences)
    return cleaned_text

def clean_description(text):
    if pd.isna(text):
        return ''
    text = remove_duplicates(text)
    text = remove_marketing_phrases(text)
    text = text.replace("!", "")
    text = text.replace("..", ".")
    return text

df = pd.read_csv('/content/drive/MyDrive/Zepto/flipkart_com-ecommerce_sample.csv')
df = df.dropna(how='all')

df['description'] = df["description"].apply(clean_description)
print(df['description'].isna().sum())

df.to_csv("/content/drive/MyDrive/Zepto/cleaned_descriptions.csv", index=False)

<ipython-input-24-e9b591a1c956>:40: DeprecationWarning: Flags not at the start of the expression '(\\bfree shipping\\b)|' (truncated) but at position 366
  cleaned_text = re.sub(pattern, '', text, flags=re.IGNORECASE)
<ipython-input-24-e9b591a1c956>:40: DeprecationWarning: Flags not at the start of the expression '(\\bfree shipping\\b)|' (truncated) but at position 407
  cleaned_text = re.sub(pattern, '', text, flags=re.IGNORECASE)
<ipython-input-24-e9b591a1c956>:40: DeprecationWarning: Flags not at the start of the expression '(\\bfree shipping\\b)|' (truncated) but at position 450
  cleaned_text = re.sub(pattern, '', text, flags=re.IGNORECASE)
<ipython-input-24-e9b591a1c956>:40: DeprecationWarning: Flags not at the start of the expression '(\\bfree shipping\\b)|' (truncated) but at position 490
  cleaned_text = re.sub(pattern, '', text, flags=re.IGNORECASE)
<ipython-input-24-e9b591a1c956>:40: DeprecationWarning: Flags not at the start of the expression '(\\bfree shipping\\b)|' (trunc

0


In [28]:
import pandas as pd
from collections import Counter
import nltk
from nltk.corpus import stopwords
import plotly.express as px

nltk.download('stopwords')
df = pd.read_csv('/content/drive/MyDrive/Zepto/cleaned_descriptions.csv')
all_descriptions = " ".join(df['description'].dropna().astype(str))

words = all_descriptions.lower().split()
stop_words = set(stopwords.words('english'))
stop_words.update(['.','-'])
words = [word for word in words if word not in stop_words]

word_counts = Counter(words)

top_words = word_counts.most_common(100)
print(len(top_words))
df_words = pd.DataFrame(top_words, columns=['Word', 'Count'])

fig = px.bar(df_words, x='Word', y='Count',
             labels={'x': 'Word', 'y': 'Frequency'},
             title='Top 100 Most Common Words in Descriptions')

fig.update_layout(xaxis_tickangle=45,
                  xaxis={'categoryorder':'total descending'},
                  xaxis_tickmode='linear',
                  xaxis_tickfont=dict(size=8))
fig.write_html("/content/drive/MyDrive/Zepto/common-words-after.html")
fig.show()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


100
